In [10]:
import numpy as np
import pandas as pd
from hazm import *
from typing import Tuple

## cleaning data

In [11]:
df = pd.read_csv('base_responses.csv')
df.head()

,Class1-Q,Class1-W,Class2-Q,Class2-W,Class3-Q,Class3-W,Class4-Q,Class4-W
0,آیا فردا هوا ابری است؟,آیا - هوا - فردا - ابری,چه زمانی اذان مغرب شروع می‌شود؟,زمان - اذان - مغرب,ساعت چنده؟,ساعت,میلاد پیامبر چه زمانی است؟,میلاد - پیامبر - زمان
1,هوا فردا آلودست ؟\nشاخص آلودگی چنده؟\nیعنی فرد...,برف - بارون - آلودگی - فردا - هواشناسی,اذان کیه؟\nطلوع آفتاب ساعت چنده؟\nاذان ظهرو گف...,اذان- غروب- افطار-طلوع-ظهر,الان تو مونیخ ساعت چنده؟\nاختلاف زمانیمون با ا...,شب - روز - ساعت - اختلاف زمانی - الان,امروز چندمه؟\nچند روز مونده تا عید؟\nوقت دکترت...,چند شنبه-تعطیلی-روز-چندمه؟-وقت
2,شب است.,شب,قبل از شب میباشد,قبل - شب,الان ۲۳ و ۲۳ دقیقه است.,۲۳ - ۲۳,چهارشنبه ۸ آذر,۴ - ۸ - ۹
3,آلودگی هوای تهران به چه علت است؟\nهوای نسبی ای...,هوا - تهران - ایران - دما - باران,اذان صبح چه ساعتی‌ست؟\nاختلاف زمانی اذان ظهر ش...,اذان - صبح - ظهر - تهران - مشهد - اختلاف,اختلاف زمانی ساعت ایران و ترکیه چقدر است؟\nامر...,اختلاف - ایران - ترکیه - امروز - سال - ۱۴۰۰,تاریخ وفات امام خمینی(ره) چه روزی است؟\nتاریخ ...,وفات - خمینی - تاریخ - شاه - خامنه‌ای - رهبر -...
4,آیا فردا هوا ابریست؟\nآیا فردا باران میبارد؟!\...,هوا-هواشناسی- امروز- فردا - باران- ابر,NaN,NaN,چند ساعت تا غروب مانده ؟!\nتهران چند ساعت با ک...,الان- زمان- ساعت,امروز چندم ماه است؟\nاین ماه چند روز دارد؟\nتا...,ماه- چندم- تاریخ


In [12]:
splitters = ['-', '_', '\n', '//', '/', 'ـ', '?', '؟']
unused_chars = ['🤤', 'ء', '(', ')', '-', '!', '.',
                '،', '_', '\n', '//', '/', 'ـ', '?', '؟']

In [13]:
norm = Normalizer()

In [14]:
def clear_class(w: pd.Series) -> list:
    # Clears the sentenses in each class
    w = w.dropna()
    for c in splitters:
        w = w.str.replace(c, "%", regex=True)
    w = w.str.split("%")
    w = np.concatenate(w.to_numpy()).ravel()
    w = np.array([norm.normalize(s) for s in w])
    for c in unused_chars:
        w = np.array([s.replace(c, '').strip() for s in w])
    w = w[w != '']
    w = w[w != ' ']
    w = w[w != '\u200c']
    return w.tolist()


# passes this test:
# full = ' '.join(w)
# ' '.join(set(full)), len(' '.join(set(full)).split())

In [15]:
w = clear_class(df["Class1-Q"])
rt = clear_class(df["Class2-Q"])
t = clear_class(df["Class3-Q"])
d = clear_class(df["Class4-Q"])

In [16]:
def remove_sents(sents: list, c: int) -> Tuple[list, list]:
    # Clears the non-relevant sentences
    idx_rm = []

    if c == 1:
        idx_rm = [6, 19, 21, 16, 18, 23, 35, 38, 43, 44, 46, 47, 58, 62, 70, 71,
                  81, 97, 126, 135, 136, 139, 151, 160, 164, 178, 225, 240, 241, 263]
    elif c == 2:
        idx_rm = [6, 12, 20, 28, 40, 45, 46, 47, 49, 50, 56,
                  76, 114, 120, 119, 144, 148, 151, 156, 173, 174]
    elif c == 3:
        idx_rm = [12, 16, 19, 20, 21, 22, 26, 30, 33, 34,
                  36, 37, 40, 41, 42, 43, 46, 47, 57, 77,
                  79, 88, 89, 96, 99, 100, 102, 101, 104,
                  103, 107, 111, 112, 113, 115, 116, 117, 118,
                  150, 155, 156, 168, 173, 174, 175, 176, 187,
                  191, 208]
    elif c == 4:
        idx_rm = [3, 16, 6, 20, 25, 30, 43, 42, 55, 88, 89, 90,
                  104, 105, 127, 132, 176, 193, 215]

    sents_rm = [sents[i] for i in idx_rm]
    sents_needed = [sents[i] for i in range(len(sents)) if i not in idx_rm]
    return sents_needed, sents_rm

In [17]:
w, w_rm = remove_sents(w, 1)
rt, rt_rm = remove_sents(rt, 2)
t, t_rm = remove_sents(t, 3)
d, d_rm = remove_sents(d, 4)

In [18]:
unk = w_rm + rt_rm + t_rm + d_rm
all_classes = [0] * len(w) + [1] * len(rt) + [2] * \
    len(t) + [3] * len(d) + [4] * len(unk)
df = pd.DataFrame({"sentence": w + rt + t + d + unk, "class": all_classes})
df.to_csv("mh_clean.csv")

# Azin's Work

In [ ]:
def clean_data(series):
    final = []
    for elem in series:
        try:
            sents = elem.split('؟')
        except:
            print("problem splitting data : " + str(elem))
        for sent in sents:
            sent = sent.replace('!', '').replace(
                '//', '').replace('/', '').replace(',', '').replace('-', '').strip()
            if(sent != ""):
                final.append(sent.strip('/n'))

    return final

In [ ]:
weather = answers['weather']

In [ ]:
final_weather = clean_data(weather)

problem splitting data : nan
problem splitting data : nan
problem splitting data : nan
problem splitting data : nan
problem splitting data : nan


In [ ]:
shari = answers['shari']

In [ ]:
final_shari = clean_data(shari)

problem splitting data : nan
problem splitting data : nan
problem splitting data : nan
problem splitting data : nan
problem splitting data : nan
problem splitting data : nan
problem splitting data : nan
problem splitting data : nan
problem splitting data : nan


In [ ]:
time = answers['time']

In [ ]:
final_time = clean_data(time)

problem splitting data : nan
problem splitting data : nan
problem splitting data : nan
problem splitting data : nan
problem splitting data : nan
problem splitting data : nan
problem splitting data : nan
problem splitting data : nan
problem splitting data : nan
problem splitting data : nan


In [ ]:
calendar = answers['calnedar']

In [ ]:
final_calendar = clean_data(calendar)

problem splitting data : nan
problem splitting data : nan
problem splitting data : nan
problem splitting data : nan
problem splitting data : nan
problem splitting data : nan
problem splitting data : nan
problem splitting data : nan
problem splitting data : nan
problem splitting data : nan


In [ ]:
classes = [1] * len(final_weather)

In [ ]:
classes.extend([2] * len(final_shari))
classes.extend([3] * len(final_time))
classes.extend([4] * len(final_calendar))

In [ ]:
final_weather.extend(final_shari)
final_weather.extend(final_time)
final_weather.extend(final_calendar)

In [ ]:
data = final_weather

In [ ]:
d = {'sentense': data,
     'class': classes}

In [ ]:
df = pd.DataFrame(d)

In [ ]:
df.to_csv('answers_clean.csv')